In [1]:
import torch
from utils import get_embedding
import pandas as pd
from tqdm import tqdm
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv

# 这个文件2个作用
## 1. 前半部分是用来获取embedding模型,然后对每个标题获取嵌入
## 2. 后半部分是针对每个标题的嵌入,计算两两余弦相似度.选取threshold,构建邻接矩阵,保存到csv文件中

In [7]:
# embedding = get_embedding(model_name="bge-large")

获取的嵌入模型是: bge-large


In [8]:
# fn = "thucnews/thucnews.csv"
# df = pd.read_csv(fn)
# df.shape

(3163, 10)

In [9]:
# title_embeddings = torch.zeros(df.shape[0], 1024)
# title_embeddings.shape

torch.Size([3163, 1024])

In [10]:
# titles = df['title']
# with torch.no_grad():
#     i = 0
#     for title in tqdm(titles):
#         t_em = torch.tensor(embedding.embed_query(title), dtype=float)
#         title_embeddings[i, :] = t_em
#         i = i + 1
#     print(i)

  0%|          | 0/3163 [00:00<?, ?it/s]

100%|██████████| 3163/3163 [08:36<00:00,  6.12it/s]

3163


In [11]:
# with open("thucnews/bge_large/pkl/title_embeddings_bge_large.pickle", "wb") as file:
#     pickle.dump(obj=title_embeddings, file=file)

# -----------------------分割线-----------------------

In [64]:
with open("thucnews/m3e_base/pkl/title_embeddings_m3e_base.pickle", "rb") as file:
    title_embeddings = pickle.load(file)

In [65]:
title_embeddings.shape

torch.Size([3163, 768])

In [66]:
score = cosine_similarity(title_embeddings, title_embeddings)

In [88]:
score = cosine_similarity(title_embeddings, title_embeddings)
threshold = 0.85
percentage = np.sum(score > threshold ) / (score.size)
print("percentage: ", percentage)
print("np.sum(score >= threshold):", np.sum(score >= threshold))
score[score < threshold] = 0
np.fill_diagonal(score, 1.0)

percentage:  0.0006809888561916061
np.sum(score >= threshold): 6813


In [89]:
save_path = f"thucnews/m3e_base/thucnews_title_{threshold}.csv"
# 打开CSV文件
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入表头
    writer.writerow(['Row', 'Column', 'Value'])

    # 遍历数组并写入CSV文件
    for i in tqdm(range(score.shape[0])):
        for j in range(score.shape[1]):
            if score[i][j] > 0:
                writer.writerow([i, j, score[i, j]])

  0%|          | 0/3163 [00:00<?, ?it/s]

100%|██████████| 3163/3163 [00:10<00:00, 297.33it/s]


In [90]:
df = pd.read_csv(save_path)
df.shape

(6813, 3)

In [91]:
df.columns

Index(['Row', 'Column', 'Value'], dtype='object')

In [92]:
df.to_csv(save_path, sep=" ", header=False, index=False)